In [1]:
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import sys
import os
import ast
from tabulate import tabulate
from pprint import pprint
from IPython.display import clear_output
import time


In [34]:
# df = pd.read_csv('../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Bangli_SUMMARIZED.csv')
df = pd.read_csv('../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Buleleng_SUMMARIZED.csv')

# df = pd.read_csv('../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Gianyar_SUMMARIZED.csv')
# df = pd.read_csv('../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Jembrana_SUMMARIZED.csv')
# df = pd.read_csv('../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Karangasem_SUMMARIZED.csv')
# df = pd.read_csv('../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Klungkung_SUMMARIZED.csv')
# df = pd.read_csv('../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Tabanan_SUMMARIZED.csv')
# df = pd.read_csv('../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Jakarta_Utara_SUMMARIZED.csv')
# df = pd.read_csv('../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Jakarta_Timur_SUMMARIZED.csv')


In [35]:
df

,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan,summarized_dakwaan,extracted_pasal,keterangan
0,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN. PERTAMA. - Bahwa Terdakwa KETUT SUBAG...,Terdakwa KETUT SUBAGIA bersama dengan Terdakwa...,['Pasal 303 Ayat (1) ke-1 KUHP jo. Pasal 2 UU ...,0
1,118/Pid.Sus/2024/PN Sgr,Narkotika,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN. KESATU. Bahwa ia Terdakwa I NYOMAN SU...,"Terdakwa I NYOMAN SUARTA pada hari Sabtu, 01 J...",['Pasal 114 Ayat (1) UU RI No. 35 Tahun 2009 T...,0
2,114/Pid.Sus/2024/PN Sgr,Narkotika,['RICO JAYADI'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['RICO JAYADI~Subsider Penjara (3 Bulan )\nPid...,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN. KESATU. Bahwa ia Terdakwa RICO JAYADI...,"Terdakwa RICO JAYADI pada hari Rabu, 29 Mei 20...",['Pasal 114 Ayat (2) UU RI No. 35 Tahun 2009 T...,0
3,115/Pid.Sus/2024/PN Sgr,Narkotika,['SANG PUTU WIDIANA'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['SANG PUTU WIDIANA~Subsider Penjara (2 Bulan ...,1(satu) buah HP merk Realme warna hitam putih ...,Dakwaan. Ke Satu. - Bahwa terdakwa SANG PUTU W...,Terdakwa SANG PUTU WIDIANA bersama-sama dengan...,['0'],1
4,116/Pid.Sus/2024/PN Sgr,Narkotika,['WAYAN DARNA MASTONO'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['WAYAN DARNA MASTONO~Subsider Penjara (2 Bula...,6 (enam) Paket plastik klip bening yang berisi...,dakwaan petugas Sat Narkoba Polres Buleleng me...,Terdakwa SANG PUTU WIDIANA bersama-sama dengan...,['Pasal 114 Ayat (1) UU RI No. 35 Tahun 2009 T...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
931,239/Pid.B/2017/PN Sgr,Pencurian,['Nyoman Sukerta Yasa'],"['I KETUT KINDRA, SH']",Mayasari Oktavia,3,['Nyoman Sukerta Yasa~Pidana Penjara Waktu Ter...,1 ( satu ) unit sepeda motor Honda Vario warna...,"- Bahwa terdakwa NYOMAN SUKERTA YASA, pada har...","Terdakwa Nyoman Sukerta Yasa, pada hari Sabtu,...",['Pasal 363 Ayat (1) ke-5 KUHP'],0
932,237/Pid.B/2017/PN Sgr,Penggelapan,['Ayu Putu Erlyandani'],"['IMAM EKA SETYAWAN, SH.']",Ni Made Dewi Sukrani,4,['Ayu Putu Erlyandani~Pidana Penjara Waktu Ter...,1 (satu) buah Buku Simpanan atau Tabungan Kopr...,Kesatu. Primair. - Bahwa terdakwa AYU PUTU ERL...,"Terdakwa AYU PUTU ERLYANDANI, yang bekerja seb...",['Pasal 374 KUHP'],0
933,236/Pid.B/2017/PN Sgr,Kejahatan terhadap Nyawa,['Ketut Mahardika Alias Kelet'],"['I Nyoman Sulitra, SH.,MH.']",Ni Made Dewi Sukrani,10,['Ketut Mahardika Alias Kelet~Pidana Penjara W...,"1 (satu) buah pedang dengan panjang 55 cm, den...",Kesatu. - Bahwa terdakwa KETUT MAHARDIKA Als K...,NaN,[],1
934,235/Pid.B/2017/PN Sgr,Pencurian,['Gunawan Saputra Alias Gunawan'],"['I Nyoman Sulitra, SH.,MH.']",Ida Ayu Sri Adriyanthi Astuti Widja,3,['Gunawan Saputra Alias Gunawan~Pidana Penjara...,- 1 (satu) unit Kendaraan Honda Vario warna Hi...,- Bahwa terdakwa I Gunawan Saputra Als. Gunawa...,"Terdakwa I, Gunawan Saputra alias Gunawan, dan...",['Pasal 363 Ayat (1) ke-4 KUHP'],0


### FUNCTION

In [36]:
barang_bukti_new_path = f'../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Buleleng_SUMMARIZED_NEW_BARANG_BUKTI.csv'

if os.path.exists(barang_bukti_new_path):
    print("Previous barang bukti summarization data exist!")
    df_summarized = pd.read_csv(barang_bukti_new_path)

elif 'summarized_barang_bukti' not in df.columns:
    print("creating summarized_barang_bukti column...")
    df_summarized = df.copy()

    df_summarized['summarized_barang_bukti'] = ''
    df_summarized['summarized_barang_bukti'] = df_summarized['summarized_barang_bukti'].astype(str)
    df_summarized['keterangan_barang_bukti'] = 1
    

    df_summarized.to_csv(barang_bukti_new_path, index=False)
    print(f"New CSV Created! -> {barang_bukti_new_path}")
    
    df_summarized = pd.read_csv(barang_bukti_new_path)

else:
    print("Error!")

creating summarized_barang_bukti column...
New CSV Created! -> ../Data/STAGE 2 PREPROCESSING/Summarized/STAGE_2_PREPROCESSING_Buleleng_SUMMARIZED_NEW_BARANG_BUKTI.csv


In [37]:
df_summarized

,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan,summarized_dakwaan,extracted_pasal,keterangan,summarized_barang_bukti,keterangan_barang_bukti
0,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN. PERTAMA. - Bahwa Terdakwa KETUT SUBAG...,Terdakwa KETUT SUBAGIA bersama dengan Terdakwa...,['Pasal 303 Ayat (1) ke-1 KUHP jo. Pasal 2 UU ...,0,NaN,1
1,118/Pid.Sus/2024/PN Sgr,Narkotika,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN. KESATU. Bahwa ia Terdakwa I NYOMAN SU...,"Terdakwa I NYOMAN SUARTA pada hari Sabtu, 01 J...",['Pasal 114 Ayat (1) UU RI No. 35 Tahun 2009 T...,0,NaN,1
2,114/Pid.Sus/2024/PN Sgr,Narkotika,['RICO JAYADI'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['RICO JAYADI~Subsider Penjara (3 Bulan )\nPid...,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN. KESATU. Bahwa ia Terdakwa RICO JAYADI...,"Terdakwa RICO JAYADI pada hari Rabu, 29 Mei 20...",['Pasal 114 Ayat (2) UU RI No. 35 Tahun 2009 T...,0,NaN,1
3,115/Pid.Sus/2024/PN Sgr,Narkotika,['SANG PUTU WIDIANA'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['SANG PUTU WIDIANA~Subsider Penjara (2 Bulan ...,1(satu) buah HP merk Realme warna hitam putih ...,Dakwaan. Ke Satu. - Bahwa terdakwa SANG PUTU W...,Terdakwa SANG PUTU WIDIANA bersama-sama dengan...,['0'],1,NaN,1
4,116/Pid.Sus/2024/PN Sgr,Narkotika,['WAYAN DARNA MASTONO'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['WAYAN DARNA MASTONO~Subsider Penjara (2 Bula...,6 (enam) Paket plastik klip bening yang berisi...,dakwaan petugas Sat Narkoba Polres Buleleng me...,Terdakwa SANG PUTU WIDIANA bersama-sama dengan...,['Pasal 114 Ayat (1) UU RI No. 35 Tahun 2009 T...,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,239/Pid.B/2017/PN Sgr,Pencurian,['Nyoman Sukerta Yasa'],"['I KETUT KINDRA, SH']",Mayasari Oktavia,3,['Nyoman Sukerta Yasa~Pidana Penjara Waktu Ter...,1 ( satu ) unit sepeda motor Honda Vario warna...,"- Bahwa terdakwa NYOMAN SUKERTA YASA, pada har...","Terdakwa Nyoman Sukerta Yasa, pada hari Sabtu,...",['Pasal 363 Ayat (1) ke-5 KUHP'],0,NaN,1
932,237/Pid.B/2017/PN Sgr,Penggelapan,['Ayu Putu Erlyandani'],"['IMAM EKA SETYAWAN, SH.']",Ni Made Dewi Sukrani,4,['Ayu Putu Erlyandani~Pidana Penjara Waktu Ter...,1 (satu) buah Buku Simpanan atau Tabungan Kopr...,Kesatu. Primair. - Bahwa terdakwa AYU PUTU ERL...,"Terdakwa AYU PUTU ERLYANDANI, yang bekerja seb...",['Pasal 374 KUHP'],0,NaN,1
933,236/Pid.B/2017/PN Sgr,Kejahatan terhadap Nyawa,['Ketut Mahardika Alias Kelet'],"['I Nyoman Sulitra, SH.,MH.']",Ni Made Dewi Sukrani,10,['Ketut Mahardika Alias Kelet~Pidana Penjara W...,"1 (satu) buah pedang dengan panjang 55 cm, den...",Kesatu. - Bahwa terdakwa KETUT MAHARDIKA Als K...,NaN,[],1,NaN,1
934,235/Pid.B/2017/PN Sgr,Pencurian,['Gunawan Saputra Alias Gunawan'],"['I Nyoman Sulitra, SH.,MH.']",Ida Ayu Sri Adriyanthi Astuti Widja,3,['Gunawan Saputra Alias Gunawan~Pidana Penjara...,- 1 (satu) unit Kendaraan Honda Vario warna Hi...,- Bahwa terdakwa I Gunawan Saputra Als. Gunawa...,"Terdakwa I, Gunawan Saputra alias Gunawan, dan...",['Pasal 363 Ayat (1) ke-4 KUHP'],0,NaN,1


In [38]:
def simply_summary(df):
    index_flag = True

    try:
        df['summarized_barang_bukti'] = df['summarized_barang_bukti'].astype('object')

        for index, row in df.iterrows():
            if row['keterangan'] != row['keterangan_barang_bukti']:
                print(f"{index}. {row['nomor_perkara']} - \t Status: {row['keterangan']}")
                
                pprint(f"Barang bukti : {row['cleaned_barang_bukti']}", width=180)
                pd.Series([row['cleaned_barang_bukti']]).to_clipboard(index=False, header=False)

                sys.stdout.flush()

                summarized_barang_bukti = input("Ringkasan Barang Bukti: ")

                if summarized_barang_bukti == "":
                    print("\nRingkasan Barang Bukti value is empty!")
                    break

                df.at[index, 'summarized_barang_bukti'] = summarized_barang_bukti
                df.at[index, 'keterangan_barang_bukti'] = 0
                print("------------------------------------------------\nSummarized Barang Bukti updated!\n")

                df.to_csv(barang_bukti_new_path, index=False)
                print("------------------------------------------------\nSummarized Barang Bukti saved to CSV!")

                index_flag = False
                # break
            
    except Exception as e:
        print(f"Error: {e} -> {index}")
    
    if index_flag:
        print("------------------------------------------------\nEnd of DataFrame")

In [40]:
simply_summary(df_summarized)

11. 102/Pid.B/2024/PN Sgr - 	 Status: 0
('Barang bukti : 1 (satu) buah HP (Handphone) Merek Poco X3 GT, Warna Hitam denga No. IME 1 863957050848743, No. IME 2 863957050848750 Dikembalikan kepada yang berhak yakni '
 'kepada Saksi Korban NYOMAN ARI SATRIAWAN Membebani terdakwa untuk membayar biaya perkara sejumlah Rp5000,00 (Lima ribu rupiah)')
------------------------------------------------
Summarized Barang Bukti updated!

------------------------------------------------
Summarized Barang Bukti saved to CSV!
12. 101/Pid.B/2024/PN Sgr - 	 Status: 0
('Barang bukti : 1 (satu) buah gelang emas (batu ungu) berat 5 gram 1 (satu) buah gelang emas bangle berat 3,5 gram 1 (satu) buah gelang rantai emas berat 9,8 gram 1 (satu) buah '
 'cincin emas mata Batu merah berat 2,5 gram 2 (dua) pasang anting-anting emas 1 (satu) buah dompet warna merah muda dengan motif kupu-kupu 1 (satu) lembar nota dari toko '
 'perhiasan emas Dewata Ayu tanggal 10 Oktober 2022 dengan nota No. KP 185. 1 (satu) kalung

In [42]:
df_test = df_summarized[['nomor_perkara','cleaned_barang_bukti','summarized_barang_bukti']].copy()

In [44]:
df_test.to_excel('../Data/STAGE 2 PREPROCESSING/Summarized/GEMINI_STAGE_2_PREPROCESSING_Buleleng_SUMMARIZED_NEW_BARANG_BUKTI.xlsx', index=False)